In [61]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import datetime
import json
import numpy as np

from utils import *

In [62]:
baseFile = '../DataExplore/eegPattern58.csv'
base = pd.read_csv(baseFile)
base = base.drop_duplicates(keep='first')

In [63]:
print(base)
base.to_csv('filteredBase.csv', index=False)

    kurtosis       mean  meanAmplitude  meanFrequency    median  \
0    11.5723  -0.310300        0.64558       0.084093 -0.615560   
1     7.0162  -0.314940        0.20093       0.051268 -0.827660   
2     3.6025  -1.321800        1.23030       0.036277 -2.901600   
3     3.9980   1.023900        5.20380       0.045542 -0.903780   
4     3.1758  -0.382250        2.81450       0.041551 -1.252400   
5     3.0161  -0.920910        2.43670       0.035781  0.071747   
6     4.0278   0.207930        2.58920       0.039401 -1.106400   
7     3.4624  -0.411120        6.65280       0.048936 -2.588700   
8     3.2687  -0.966290        2.50330       0.048646 -0.005126   
9     3.2905  -0.578260        4.59210       0.046684 -0.954440   
10    5.2384  -0.637670        2.60540       0.028987 -3.665500   
11    3.6853  -1.867000        2.40830       0.032303 -5.098200   
12    3.6137   0.556660        4.09630       0.053726 -1.596900   
13    3.3334   0.020575        3.51950       0.055367 -1.21110

In [64]:
features = base.drop(columns=['zzclassification'])
classes = base.loc[:, ["zzclassification"]].values
featureList = "kurtosis,mean,meanAmplitude,meanFrequency,median,medianFrequency,skewness,totalPower,variance,zzclassification,meandelta,meanAmplitudedelta,powerdelta,meanAdelta,meanFidelta,meanalpha,meanAmplitudealpha,poweralpha,meanAalpha,meanFialpha,meantheta,meanAmplitudetheta,powertheta,meanAtheta,meanFitheta,meanbeta,meanAmplitudebeta,powerbeta,meanAbeta,meanFibeta,meangamma,meanAmplitudegamma,powergamma,meanAgamma,meanFigamma".split(",")

exclusionList = "zzclassification,mean,meanFrequency,medianFrequency,meandelta,meantheta,meangamma,meanAmplitudegamma,meanFigamma,meanAgamma,meanbeta"
for i in exclusionList.split(","):
    if i in featureList:
        featureList.remove(i)
print(featureList)
features = selectFeature(features=features, featureList=featureList)

['kurtosis', 'meanAmplitude', 'median', 'skewness', 'totalPower', 'variance', 'meanAmplitudedelta', 'powerdelta', 'meanAdelta', 'meanFidelta', 'meanalpha', 'meanAmplitudealpha', 'poweralpha', 'meanAalpha', 'meanFialpha', 'meanAmplitudetheta', 'powertheta', 'meanAtheta', 'meanFitheta', 'meanAmplitudebeta', 'powerbeta', 'meanAbeta', 'meanFibeta', 'powergamma']


In [65]:
len(featureList)

24

In [66]:
labelEncoder_prev = LabelEncoder()
classes = labelEncoder_prev.fit_transform(classes)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [67]:
trainingInputs, testInputs, trainingClasses, testClasses = train_test_split(features, classes, test_size=0.25)

In [68]:
classifier = KerasClassifier(build_fn=createNetwork)
parameters = {
    'batch_size': [35, 50],
    'epochs': [2000, 5000],
    'inputShape': [(len(featureList),)],
    'hiddenLayerUnits': [[14], [12], [round(len(featureList)/2)]],
    'kernel_initializer': ['random_uniform', 'normal'],
    'optimizer': ['adamax'],
    'loss': ['binary_crossentropy'],
    
}

C:\Users\Greze\AppData\Local\Temp\ipykernel_14364\2099484745.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=createNetwork)


In [69]:
base

,kurtosis,mean,meanAmplitude,meanFrequency,median,medianFrequency,skewness,totalPower,variance,zzclassification,...,meanbeta,meanAmplitudebeta,powerbeta,meanAbeta,meanFibeta,meangamma,meanAmplitudegamma,powergamma,meanAgamma,meanFigamma
0,11.5723,-0.310300,0.64558,0.084093,-0.615560,0.031330,1.245500,2494.0973,60.8402,TA,...,-0.000459,1.00790,48.0378,1.3049,13.1943,-0.000324,0.69897,24.3555,1.08950,13.6041
1,7.0162,-0.314940,0.20093,0.051268,-0.827660,0.024136,1.058400,2783.4228,67.9031,TA,...,-0.000803,0.95488,30.2312,1.2011,12.4959,-0.000505,0.32039,5.5139,0.54754,8.4207
2,3.6025,-1.321800,1.23030,0.036277,-2.901600,0.018814,0.414100,21548.0296,525.0551,TA,...,0.005447,2.56130,211.7835,3.2146,33.2020,0.003836,0.72781,29.7440,1.14070,18.4952
3,3.9980,1.023900,5.20380,0.045542,-0.903780,0.023091,0.379710,28025.9247,683.6762,TA,...,-0.001976,2.91680,318.9589,3.7166,38.5210,-0.000680,0.79973,51.7185,1.25720,20.4667
4,3.1758,-0.382250,2.81450,0.041551,-1.252400,0.025062,0.238260,34058.7570,831.5586,HVS,...,0.004172,2.84270,270.4779,3.5209,35.8646,0.002316,0.73932,23.7281,1.08930,18.2635
5,3.0161,-0.920910,2.43670,0.035781,0.071747,0.018653,-0.099103,32499.7257,793.0450,TA,...,0.000487,2.29000,153.7662,2.8373,29.4249,0.000366,0.84970,26.4419,1.18680,19.7405
6,4.0278,0.207930,2.58920,0.039401,-1.106400,0.021885,-0.007763,29075.2762,709.9365,TA,...,0.003262,2.44380,203.4739,3.0047,31.5696,0.001558,0.85993,27.3842,1.20280,20.3022
7,3.4624,-0.411120,6.65280,0.048936,-2.588700,0.026128,0.175620,43640.7592,1065.5187,HVS,...,0.008108,3.11100,377.5845,4.0209,39.1651,0.005555,0.73157,39.7811,1.09680,17.8911
8,3.2687,-0.966290,2.50330,0.048646,-0.005126,0.026185,0.074128,34766.2165,848.3343,HVS,...,-0.003302,2.45260,220.6215,3.2307,31.9142,-0.002830,0.89553,46.4522,1.27390,20.4739
9,3.2905,-0.578260,4.59210,0.046684,-0.954440,0.024200,-0.035649,30627.5691,747.6561,HVS,...,-0.003083,2.49370,254.6167,3.3056,32.0270,-0.001848,0.69258,22.2593,1.09210,17.6776


In [70]:
gridSearch = GridSearchCV(
    estimator=classifier,
    param_grid=parameters,
    scoring='accuracy',
    cv=5    
)

gridSearch = gridSearch.fit(
    trainingInputs,
    trainingClasses,
    verbose=0,
    use_multiprocessing = True
)

1/1 [==============================] - 0s 55ms/step


In [71]:
best_params = gridSearch.best_params_
best_accuracy = gridSearch.best_score_

In [72]:
logFile = 'modelLog58.json'

execution_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
accuracy = best_accuracy
params = best_params
append_log(execution_date, accuracy, params, featureList, logFile, baseFile)

In [73]:
print(best_accuracy)
print(best_params)

0.6555555555555556
{'batch_size': 35, 'epochs': 5000, 'hiddenLayerUnits': [12], 'inputShape': (24,), 'kernel_initializer': 'normal', 'loss': 'binary_crossentropy', 'optimizer': 'adamax'}


In [74]:
l1 = [
            "kurtosis",
            "mean",
            "meanAmplitude",
            "meanFrequency",
            "median",
            "medianFrequency",
            "skewness",
            "totalPower",
            "variance",
            "meanAmplitudedelta",
            "powerdelta",
            "meanAdelta",
            "meanFidelta",
            "meanAmplitudealpha",
            "poweralpha",
            "meanAalpha",
            "meanFialpha",
            "meanAmplitudetheta",
            "powertheta",
            "meanAtheta",
            "meanFitheta",
            "meanAmplitudebeta",
            "powerbeta",
            "meanAbeta",
            "meanFibeta",
            "powergamma"
        ]


l2 = [
            "kurtosis",
            "mean",
            "meanAmplitude",
            "median",
            "skewness",
            "totalPower",
            "variance",
            "meandelta",
            "meanAmplitudedelta",
            "powerdelta",
            "meanAdelta",
            "meanFidelta",
            "meanalpha",
            "meanAmplitudealpha",
            "poweralpha",
            "meanAalpha",
            "meanFialpha",
            "meantheta",
            "meanAmplitudetheta",
            "powertheta",
            "meanAtheta",
            "meanFitheta",
            "meanbeta",
            "meanAmplitudebeta",
            "powerbeta",
            "meanAbeta",
            "meanFibeta",
            "powergamma"
        ]



difference = [item for item in l2 if item not in l1]
difference2 = [item for item in l1 if item not in l2]

In [75]:
difference
print(len(l1), len(l2))
print(difference)
print(difference2)

26 28
['meandelta', 'meanalpha', 'meantheta', 'meanbeta']
['meanFrequency', 'medianFrequency']
